# Lab for loading and transforming the data
Responsible: Fabian (01126693)

## Task

As group 5 we have chosen task 6. Task 6 deals with natural disasters and poses 3 questions to be answered:

1. How did the number of deaths per year from natural disasters change over the last hundred years? 
2. How does this vary by country? How does this vary by type of natural disaster?  
3. Are there trends visible that could be due to climate change? 

Based on the questions posed, some requirements for the data set emerge. Question 1 aims at the change in death counts over the last 100 years. Therefore, the dataset must contain death counts as well as a history as long as possible (optimally in the range of 100 years). Question 2 focuses on the development in different countries and whether there are differences between different types of natural disasters. Therefore, the data set must include both geographic characteristics and attributes to distinguish between different types of natural disasters. 

In [59]:
import numpy as np
import pandas as pd
import pickle

In [60]:
df_transformed = pd.DataFrame() 

raw_data = pd.read_csv("../data/data_raw.csv",
                       sep = ';')
print(raw_data.head(5))
raw_data.dtypes
df_transformed = raw_data.convert_dtypes()

  Disaster_Group Disaster_Subgroup      Disaster_Type     Country  ISO  \
0        Natural    Climatological            Drought  Cabo Verde  CPV   
1        Natural    Climatological            Drought       India  IND   
2        Natural       Geophysical         Earthquake   Guatemala  GTM   
3        Natural       Geophysical  Volcanic activity   Guatemala  GTM   
4        Natural       Geophysical  Volcanic activity   Guatemala  GTM   

            Region Continent  Start_Year  Total_Deaths  No_Affected  
0   Western Africa    Africa        1900       11000.0          NaN  
1    Southern Asia      Asia        1900     1250000.0          NaN  
2  Central America  Americas        1902        2000.0          NaN  
3  Central America  Americas        1902        1000.0          NaN  
4  Central America  Americas        1902        6000.0          NaN  


## Data Source(s)
Based on these requirements, publicly available datasets were sought. The 2 most relevant of these are presented briefly here and an explanation is given of how the final choice of dataset was made. 

### WHO data [<sup>1</sup>](#fn1) ( Explored but not used)

The data was queried from the Mortality Database provided by the WHO. With regard to the origin, completeness and scope of the data, the WHO says: 

> The WHO Mortality Database is a compilation of mortality data by country and area, year, sex, age and cause of death, as transmitted annually by national authorities from their civil registration and vital statistics system. It comprises data since 1950 to date. Only data with at least 65% completeness are published here

##### Requirements met
 - Total death counts per year 
 - Split by country and geographical region 
 - Split by sex (additional information)
 - Split by age (additional information)

##### Requirements not met
 - Long history missing: data for natural disasters starts 1979
 - Missing split by type of natural disaster
 
It can be seen that the WHO data are more detailed in terms of gender and age categories, but they have no distinction of natural disaster categories and also have a limited history of only 50 years. 

### EM-DAT data [<sup>2</sup>](#fn2) (Explored and used)

> In 1988, the Centre for Research on the Epidemiology of Disasters (CRED) launched the Emergency  Events Database (EM-DAT). EM-DAT was created with the initial support of the World Health Organisation (WHO) and the Belgian Government.
>
>The main objective of the database is to serve the purposes of humanitarian action at national and international levels. The initiative aims to rationalise decision making for disaster preparedness, as well as provide an objective base for vulnerability assessment and priority setting.
>
>EM-DAT contains essential core data on the occurrence and effects of over 22,000 mass disasters in the world from 1900 to the present day. The database is compiled from various sources, including UN agencies, non-governmental organisations, insurance companies, research institutes and press agencies.
>
> -- Source: https://www.emdat.be/

##### Requirements met
 - Total death counts per event 
 - Country, region and continent information per event
 - Events dated back till 1900
 - Disaster type per event
 
Since the questions do not require an analysis based on gender or age categories, but the EM-DAT dataset covers a much longer period (since 1900) and also assigns a category to each natural disaster, this dataset is perfectly suited for the task and therefore used for the analysis. In the following section, a detailed overview of the attributes used in the further analysis will be given. 

#### Data fields

 - `Disaster_Group`: Two different types of disasters can be distinguished in the EM-DAT database: natural disasters and technological disasters. Since we are only interested in natural disasters, only they were requested and consequently this field contains only the value *Natural*. No missing values are present for this attribute. 
 - `Disaster_Subgroup`: Every natural disaster is assigned to one of the following six subgroups: *Biological, Geophysical, Climatological, Hydrological, Meteorological and Extra-terrestrial* to describe the type of natural disaster. No missing values are present for this attribute. 
 - `Disaster_type`: For every natural disaster event one main disaster type is identified. If two or more disasters are related because they are consequences of each other, then this information is encoded in the attributes `Associated_Dis` and `Associated_Dis2`. No missing values are present for this attribute. 
 
> Types of natural disasters could be further broken down using two more categories which would be available in the database. For example, the Disaster type *Storm* could be further subdivided into *Tropical storm*, *Extra-tropical storm* or *Convective storm*. Even a further subdivision of the category *Convective storm* would be possible. Since the analysis is aimed at detecting trends on a high level, the classification of each event based on the attributes `Disaster_Subgroup` and `Disaster _Type` was considered sufficient and the further subdivisions into `Disaster Subtype` and `Disaster Subsubtype` are not considered in the analysis. For a full list of splits see table 1 (https://public.emdat.be/about).

 - `Associated_Dis`: Secondary event triggered by a natural disaster (i.e. Landslide for a flood, explosion after an earthquake, ...)
 - `Associated_Dis2`: Another secondary event triggered by a natural disaster. (i.e. Landslide for a flood, explosion after an earthquake, ...)
 
 Example: If a tsunami is triggered by an earthquake, then the attribute `Disaster_Type` would be *Earthquake*, the attribute `Disaster_Subtype` would be *Ground movement* and the attribute `Associated_Dis` would be *Tsunami/Tidal wave*.
 
 - `Country`: The country in which the disaster has occurred or had an impact. If a disaster has affected more than one country, a seperate entry is created in the database for each country affected. No missing values are present for this attribute. 
 - `ISO`: Unique 3-letter code for each country defined by ISO 3166. No missing values are present for this attribute. 
 - `Region`: The region to which the country belongs, based on the UN regional division. No missing values are present for this attribute. 
 - `Continent`: The continent to which the country belongs. No missing values are present for this attribute. 
 - `Start_Year`: The year when the disaster occurred. No missing values are present for this attribute. 
 
> For sudden-impact disasters also the month and the day are well defined and available. For disaster situations developing gradually over a longer time period (i.e. drought) with no specific start date the day attribute is empty. For our questions the exact date plays a subordinate role and therefore the year of the beginning of the disaster is completely sufficient for our analysis.    

 - `Total_Deaths`: Number of people who lost their life because the event happened plus the number of people whose whereabouts since the disaster are unknown, and presumed dead based on official figures. Missing values present for approx. 25% of all events.
 
 - `No_Affected`: Number of people which requiring immediate assistance during an emergency situation. The indicator affected is often reported and is widely used by different actors to convey the extent, impact, or severity of a disaster in non-spatial terms. In case that no values for the attribute `Total_Deaths` are available this attribute could be used as a proxy. 

We can serialize the transformed dataframe with pickle; it is better than saving to csv because it will include data types etc.

In [61]:
transformed_df_serial_path = "../data/data_transformed.pickle"

# some sanity checks
assert isinstance(df_transformed, pd.DataFrame), "\"df_transformed\" is not a pandas dataframe!"

assert len(df_transformed)!=0, "\"df_transformed\" contains no data!"

# serializing the dataframe object to a binary file
with open(transformed_df_serial_path, 'wb') as file:
    pickle.dump(df_transformed, file)

<sup>1</sup> WHO data: <span id="fn1"> https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/natural-disasters</span>

<sup>2</sup> EM-dat data: <span id="fn2"> https://public.emdat.be/data Based on the terms of use, the data can be used for this exercise:  
    
> If you are an academic organization, a university, a non-profit research institution and/or an international public organization (UN agencies, multi-lateral banks, other multi-lateral institution and national governments) and/or part of a Media agency (journalist, press agencies) with the intention to use the EM-DAT database (hereafter ‘EM-DAT’) for research, teaching or information purposes, you shall, conditional upon the acceptance of the present conditions of use, be granted free access to EM-DAT (also ‘Authorized Use’).